In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import bifacial_radiance as br
from datetime import datetime
import re

# testfolder = Path().resolve().parent.parent / 'TEMP' / 'bifacial_radiance'


testfolder = os.path.join('HPCruns', 'cchoi2', 'newcoord')

# if not os.path.exists(testfolder):
#     os.makedirs(testfolder)
    
# print ("Your simulation will be stored in %s" % testfolder)

In [2]:
# remove unwanted results, in this case all the "fronts"
filesall = os.listdir(os.path.join(testfolder)) # How do I save the path to an object so that I don't have to repeat the whole process?

# filestoclean = [e for e in filesall_origin if e.endswith('_Front.csv')]
# for cc in range(0, len(filestoclean_origin)):
#     filetoclean_origin = filestoclean_origin[cc]
#     os.remove(os.path.join(testfolder,'BARN_validation_jan18_2023','results', filetoclean_origin))

In [3]:
print(filesall)


['irr_1axis_2023-09-26_0615Ground_S1_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0615Ground_S2_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0615Ground_S3_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0615Ground_S4_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S1_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S2_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S3_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S4_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0630Ground_S5_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S1_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S2_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S3_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0645Ground_S5_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0700Ground_S1_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0700Ground_S2_Scene0_Row5_Module10.csv', 'irr_1axis_2023-09-26_0700Ground_S4_Scene0_Row5_Module

In [4]:
from datetime import datetime
import re

filestoclean = [e for e in filesall if e.endswith('_Back.csv')]
for cc in range(0, len(filestoclean)):
    filetoclean = filestoclean[cc]
    os.remove(os.path.join(testfolder, filetoclean))

filesall = os.listdir(os.path.join(testfolder))

## Read in the results
Below script reads in the results of the bifacial radiance modeling for a specific date and appends them to arrays that will then be joined into a dataframe.

In [5]:
# exampledata = br.load.read1Result(os.path.join(testfolder, 'irr_1axis_2023-09-26_0600Ground_S2_Scene0_Row5_Module10_Front.csv'))
exampledata = br.load.read1Result(os.path.join(testfolder, filesall[0]))
print(exampledata)



       x    y     z  rearZ      mattype                rearMat  Wm2Front  \
0  0.324  0.0  0.05  1.441  groundplane  a9.4.a0.PVmodule.2310    16.015   
1  0.349  0.0  0.05  1.456  groundplane  a9.4.a0.PVmodule.2310    16.045   
2  0.375  0.0  0.05  1.470  groundplane  a9.4.a0.PVmodule.2310    16.076   
3  0.400  0.0  0.05  1.485  groundplane  a9.4.a0.PVmodule.2310    16.106   
4  0.425  0.0  0.05  1.499  groundplane  a9.4.a0.PVmodule.2310    16.137   
5  0.451  0.0  0.05  1.514  groundplane  a9.4.a0.PVmodule.2310    16.167   
6  0.476  0.0  0.05  1.528  groundplane  a9.4.a0.PVmodule.2310    16.198   
7  0.502  0.0  0.05  1.543  groundplane  a9.4.a0.PVmodule.2310    16.228   
8  0.527  0.0  0.05  1.557  groundplane  a9.4.a0.PVmodule.2310    16.077   

   Wm2Back  Back/FrontRatio  rearX  rearY  
0    4.731            0.295  0.798    0.0  
1    4.708            0.293  0.598    0.0  
2    4.684            0.291  0.399    0.0  
3    4.685            0.291  0.199    0.0  
4    4.675         

prefixed = [filename for filename in filesall]
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Modeled = []
Offset = []
Position = []
inchtom = 0.0254
print('{} files in the directory'.format(filesall.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.


ind = prefixed[0].split('_')
resultsDF = br.load.read1Result(os.path.join(testfolder, prefixed[0]))
datalength = len(resultsDF)
offsetlength = (datalength - 1)/2

print(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[0])[0])
# print(resultsDF)
for ii in range (0, datalength):
    try:
        # Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[0])[0])
        print(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[0])[0])
        # Position.append(re.search(r'S(\d+)', prefixed[i])[1])
        Position.append(re.search(r'S(\d+)', prefixed[0])[1])
        # print(Position)
        # Offset.append(re.search(r'_offset_(-?\d+)', prefixed[i])[1])
        Offset.append(np.round(resultsDF['y'][ii]/0.0254).astype(int)-offsetlength)
        # print(Offset)
        # Modeled.append(resultsDF['Wm2Front'][ii])
        # print(Modeled)

        # filenamed.append(prefixed[0])
    except:
        print(" FAILED ", i, prefixed[0])
        faillist.append(prefixed[0])

print(resultsDF)

In [6]:
prefixed = [filename for filename in filesall]
distance = []
groundirrad = []
filenamed = []
faillist = []
Datetime = []
Modeled = []
x_pos = []
# center_pos = []
Offset = []
Position = []
inchtom = 0.0254
offsetinc = 0.0254
# Positions are (re.search(r'S(\d+)', prefixed[i])[1]-1)*1.2 + 0.4254
print('{} files in the directory'.format(filesall.__len__()))
print('{} groundscan files in the directory'.format(prefixed.__len__()))
i = 0  # counter to track # files loaded.

for i in range (0, len(prefixed)):
    ind = prefixed[i].split('_')
    resultsDF = br.load.read1Result(os.path.join(testfolder, prefixed[i]))
    datalength = len(resultsDF)
    offsetlength = (datalength - 1)/2
    center = ((int(re.search(r'S(\d+)', prefixed[i])[1])-1)*1.2 + 0.4254)
    for ii in range (0, datalength):
        try:
            Datetime.append(re.search("([0-9]{4}\-[0-9]{2}\-[0-9]{2}\_[0-9]{4})", prefixed[i])[0])
            # Position.append(re.search(r'S(\d+)', prefixed[i])[1])
            Position.append(re.search(r'S(\d+)', prefixed[i])[1])
            x_pos.append(resultsDF['x'][ii])
            # Offset_dist_m.append()
            # Offset.append(re.search(r'_offset_(-?\d+)', prefixed[i])[1])
            # Offset.append(np.round(resultsDF['y'][ii]/0.0254).astype(int)-offsetlength)
            # Offset.append((np.round(resultsDF['x'][ii]-((int(re.search(r'S(\d+)', prefixed[i])[1])-1)*1.2 + 0.4254))/0.0254).astype(int))
            # Offset.append((resultsDF['x'][ii]-((int(re.search(r'S(\d+)', prefixed[i])[1])-1)*1.2 + 0.4254)/0.0254).astype(int))
            Modeled.append(resultsDF['Wm2Front'][ii])

            filenamed.append(prefixed[i])
        except:
            print(" FAILED ", i, prefixed[i])
            faillist.append(prefixed[i])

results_compiled = pd.DataFrame(list(zip(Datetime, Position, x_pos, Modeled)), columns = ['Datetime', "Position", "x", "Modeled"])
# results_compiled['center_pos'] = (int(results_compiled['Position'])-1)*1.2+0.4254
results_compiled['Position'] = pd.to_numeric(results_compiled['Position'], errors='coerce')
results_compiled['Center'] = ((results_compiled['Position']-1)*1.2+0.4254)
results_compiled['Offset'] = results_compiled.apply(lambda row: round((row['x'] - row['Center'])/0.0254), axis = 1)
# results_compiled['x2'] = results_compiled.apply(lambda row: row['x'] - ((row['Position']-1)*1.2+0.4254))
# results_compiled['Position'] = results_compiled['Position'].astype(int)

results_compiled['Datetime'] = pd.to_datetime(results_compiled['Datetime'], format ='%Y-%m-%d_%H%M')
print(results_compiled)
# print((re.search(r'S(\d+)', prefixed[2])[1].astype(int)-1)*1.2 + 0.4254)
# print(int(re.search(r'S(\d+)', prefixed[i])[1]))
# print(br.load.read1Result(os.path.join(testfolder, prefixed[0]))['x'])
# print(br.load.read1Result(os.path.join(testfolder, prefixed[2]))['x'][0]-((int(re.search(r'S(\d+)', prefixed[2])[1])-1)*1.2 + 0.4254))
# print(resultsDF['x'][4]-((int(re.search(r'S(\d+)', prefixed[2])[])-1)*1.2 + 0.4254))


213 files in the directory
213 groundscan files in the directory
                Datetime  Position      x  Modeled  Center  Offset
0    2023-09-26 06:15:00         1  0.324   16.015  0.4254      -4
1    2023-09-26 06:15:00         1  0.349   16.045  0.4254      -3
2    2023-09-26 06:15:00         1  0.375   16.076  0.4254      -2
3    2023-09-26 06:15:00         1  0.400   16.106  0.4254      -1
4    2023-09-26 06:15:00         1  0.425   16.137  0.4254       0
...                  ...       ...    ...      ...     ...     ...
1912 2023-09-26 17:45:00         5  5.225    4.907  5.2254       0
1913 2023-09-26 17:45:00         5  5.251    4.851  5.2254       1
1914 2023-09-26 17:45:00         5  5.276    4.795  5.2254       2
1915 2023-09-26 17:45:00         5  5.302    4.759  5.2254       3
1916 2023-09-26 17:45:00         5  5.327    4.717  5.2254       4

[1917 rows x 6 columns]


In [50]:
# del(measured_l)
# Measured irradiance from Solar Radiation Research Laboratory (SRRL) at NREL
measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1).rename(columns = {"Ap_1":"1", "Ap_2":"2", "Ap_3":"3", "Ap_4":"4", "Ap_5":"5"}) # The field pyranometer data
measured['Datetime'] = pd.to_datetime(measured['TIMESTAMP'], format ='%m/%d/%Y %H:%M')
measured = measured.drop(["TIMESTAMP", "RECORD"], axis = 'columns') # drop this column so the resample command does not confuse TIMESTAMP and Datetime columns
measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='right', closed='right').mean().reset_index() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.
# print(measured)
measured_irrad = measured[["Datetime", "1", "2", "3", "4", "5"]]
measured_irrad = pd.melt(measured_irrad, id_vars = ["Datetime"], value_vars = ["1", "2", "3", "4", "5"], value_name = "Measured", var_name = "Position")
measured_irrad = pd.merge(measured_irrad, measured[["Datetime", "PAR_con", "PAR_pv"]])
measured_irrad['Datetime'] = pd.to_datetime(measured_irrad['Datetime'], format ='%m/%d/%Y %H:%M')
measured_irrad['Position'] = measured_irrad['Position'].astype(int)

# Modify the Measured column for rows where Position == 4
measured_irrad.loc[measured_irrad["Position"] == 4, "Measured"] /= 1.08
# print(measured_irrad)
# # combined = pd.concat([results_compiled, measured_irrad], axis = 0)#,keys= ["Datetime", "Position", "Offset"])#.set_index(["Datetime", "Position"])

# # combined = pd.concat([results_compiled, measured_irrad.assign(Offset = -2), measured_irrad.assign(Offset = -1), measured_irrad.assign(Offset = 0), measured_irrad.assign(Offset = 1), measured_irrad.assign(Offset = 2)], join = "outer")#,keys= ["Datetime", "Position", "Offset"])
combined = pd.merge(results_compiled, measured_irrad)
# combined = pd.merge(results_compiled, measured_irrad).drop_duplicates(subset = ["Datetime", "Position", "Offset"])#, on = ["Datetime", "Position"])
combined['ModeledMinusMeasured'] = combined['Modeled'] - combined['Measured']
print(combined)

# # results_compiled.merge()
# print(measured)
# # measured = measured.set_index('Datetime').resample('15T', axis = 'index', label='right', closed='right').mean() # The "label" and the "closed" arguments are different from that of how the SRRL data was read-in (which was "right" for both arguments), but this somehow yields a better timing match.
# # measured_l = measured.reset_index()[["Datetime",'ap1', 'ap2', 'ap3', 'ap4', 'ap5']].melt(id_vars= "Datetime", value_vars = ['ap1', 'ap2', 'ap3', 'ap4', 'ap5'], value_name = "measured")
# # measured_l_offset = pd.concat([measured_l.assign(offset = "west"), measured_l.assign(offset = "east"), measured_l.assign(offset = "origin")]).set_index(["Datetime", "variable", "offset"])

                Datetime  Position      x  Modeled  Center  Offset   Measured  \
0    2023-09-26 06:15:00         1  0.324   16.015  0.4254      -4  16.187596   
1    2023-09-26 06:15:00         1  0.349   16.045  0.4254      -3  16.187596   
2    2023-09-26 06:15:00         1  0.375   16.076  0.4254      -2  16.187596   
3    2023-09-26 06:15:00         1  0.400   16.106  0.4254      -1  16.187596   
4    2023-09-26 06:15:00         1  0.425   16.137  0.4254       0  16.187596   
...                  ...       ...    ...      ...     ...     ...        ...   
1912 2023-09-26 17:45:00         5  5.225    4.907  5.2254       0   5.515171   
1913 2023-09-26 17:45:00         5  5.251    4.851  5.2254       1   5.515171   
1914 2023-09-26 17:45:00         5  5.276    4.795  5.2254       2   5.515171   
1915 2023-09-26 17:45:00         5  5.302    4.759  5.2254       3   5.515171   
1916 2023-09-26 17:45:00         5  5.327    4.717  5.2254       4   5.515171   

        PAR_con     PAR_pv 

In [51]:

# Filter both dataframes for the specified time range
start_time = "2023-09-26 06:15:00"
end_time = "2023-09-26 17:45:00"

results_filtered = results_compiled[
    (results_compiled['Datetime'] >= start_time) & (results_compiled['Datetime'] <= end_time)
]
measured_filtered = measured_irrad[
    (measured_irrad['Datetime'] >= start_time) & (measured_irrad['Datetime'] <= end_time)
]

# Perform an outer merge on Datetime and Position
combined_filter2 = pd.merge(
    results_filtered,
    measured_filtered,
    on=['Datetime', 'Position'],
    how='outer'
)

# Sort the merged data by Datetime and Position
combined_filter2 = combined_filter2.sort_values(by=['Datetime', 'Position'])

# Display the merged dataframe
print(results_filtered)
print(measured_filtered)
print(combined_filter2)

# Load the measured data
# measured = pd.read_csv(os.path.join(os.path.join(Path().resolve(), 'Data','BARNirrad_measured.csv')), header = 1) # The field pyranometer data
srrl_weather = pd.read_csv(str(Path().resolve() / 'WeatherFiles' /  'PSM3_15T.csv'), header = 2) # Data from SRRL

# Create a Datetime variable and set it as the index
srrl_weather['date'] = srrl_weather[['Year','Month','Day']].astype(str).agg('-'.join, axis=1) # Couldn't think of a proper way to chain this process into a single command. If you think of something, please let me know
srrl_weather['time'] = srrl_weather[['Hour','Minute']].astype(str).agg(':'.join, axis=1)
srrl_weather['Datetime'] = pd.to_datetime(srrl_weather[['date','time']].agg(" ".join, axis=1))
# srrl_weather.set_index('Datetime', inplace=True)

srrl_filtered = srrl_weather[
    (srrl_weather['Datetime'] >= start_time) & (srrl_weather['Datetime'] <= end_time)
]

combined_ref = pd.merge(combined_filter2, srrl_filtered.drop(columns=['Year', 'Month', 'Day', 'Hour', 'Minute']))
# print(srrl_weather)

                Datetime  Position      x  Modeled  Center  Offset
0    2023-09-26 06:15:00         1  0.324   16.015  0.4254      -4
1    2023-09-26 06:15:00         1  0.349   16.045  0.4254      -3
2    2023-09-26 06:15:00         1  0.375   16.076  0.4254      -2
3    2023-09-26 06:15:00         1  0.400   16.106  0.4254      -1
4    2023-09-26 06:15:00         1  0.425   16.137  0.4254       0
...                  ...       ...    ...      ...     ...     ...
1912 2023-09-26 17:45:00         5  5.225    4.907  5.2254       0
1913 2023-09-26 17:45:00         5  5.251    4.851  5.2254       1
1914 2023-09-26 17:45:00         5  5.276    4.795  5.2254       2
1915 2023-09-26 17:45:00         5  5.302    4.759  5.2254       3
1916 2023-09-26 17:45:00         5  5.327    4.717  5.2254       4

[1917 rows x 6 columns]
                Datetime  Position   Measured    PAR_con     PAR_pv
9015 2023-09-26 06:15:00         1  16.187596  44.796661  45.701855
9016 2023-09-26 06:15:00         2 

In [52]:
# look at residuals (MBD, RMSE) based on Grear, Gpoa and Gtotal_modeled. From B. Marion Solar Energy 2016
def MBD(meas,model):
    # MBD=100∙[((1⁄(m)∙∑〖(y_i-x_i)]÷[(1⁄(m)∙∑〖x_i]〗)〗)
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    # rudimentary filtering of modeled irradiance
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = 100*((1/m)*sum(df.model-df.meas))/df.meas.mean()
    return out

def RMSE(meas,model):
    #RMSD=100∙〖[(1⁄(m)∙∑▒(y_i-x_i )^2 )]〗^(1⁄2)÷[(1⁄(m)∙∑▒〖x_i]〗)
    import numpy as np
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = 100*np.sqrt(1/m*sum((df.model-df.meas)**2))/df.meas.mean()
    return out

# residuals absolute output (not %) 
def MBD_abs(meas,model):
    # MBD=100∙[((1⁄(m)∙∑〖(y_i-x_i)]÷[(1⁄(m)∙∑〖x_i]〗)〗)
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    # rudimentary filtering of modeled irradiance
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = ((1/m)*sum(df.model-df.meas))
    return out

def RMSE_abs(meas,model):
    #RMSD=100∙〖[(1⁄(m)∙∑▒(y_i-x_i )^2 )]〗^(1⁄2)÷[(1⁄(m)∙∑▒〖x_i]〗)
    import numpy as np
    import pandas as pd
    df = pd.DataFrame({'model':model,'meas':meas})
    df = df.dropna()
    minirr = meas.min()
    df = df[df.model>minirr]
    m = df.__len__()
    out = np.sqrt(1/m*sum((df.model-df.meas)**2))
    return out

In [53]:
# Apply the RMSE and MBD functions by groups of position + offset
combined_mbd = combined.groupby(by = ['Position', 'Offset']).apply(lambda x: MBD(x["Modeled"], x["Measured"])).reset_index(name = "mbd")
combined_mbdabs = combined.groupby(by = ['Position', 'Offset']).apply(lambda x: MBD_abs(x["Modeled"], x["Measured"])).reset_index(name = "mbd_abs")
combined_rmse = combined.groupby(by = ['Position', 'Offset']).apply(lambda x: RMSE(x["Modeled"], x["Measured"])).reset_index(name = "rmse")
combined_rmseabs = combined.groupby(by = ['Position', 'Offset']).apply(lambda x: RMSE_abs(x["Modeled"], x["Measured"])).reset_index(name = "rmse_abs")

combined_validation = combined_mbd.merge(combined_mbdabs.merge(combined_rmse.merge(combined_rmseabs, on = ['Position', 'Offset']), on = ['Position', 'Offset']), on = ['Position', 'Offset'])
combined_validation["Offset"] = combined_validation["Offset"].astype("category")
# combined_validation["Offset"].cat.reorder_categories(["-2", "-1", "0", "1", "2"])
print(combined_validation)

    Position Offset        mbd    mbd_abs       rmse    rmse_abs
0          1     -4  12.218301  21.523250  72.810082  128.259204
1          1     -3  12.125231  21.377031  72.653682  128.089929
2          1     -2  19.436705  32.169592  65.498667  108.406508
3          1     -1  14.777940  25.451665  62.874110  108.286455
4          1      0   4.525699   8.559006  20.508686   38.786042
5          1      1   4.374547   8.285128  20.377638   38.594013
6          1      2   4.213566   7.992567  20.251880   38.415085
7          1      3  15.003262  25.789104  44.266348   76.089415
8          1      4  14.890890  25.620982  44.168059   75.994722
9          2     -4   6.052720  16.604062  40.135528  110.101375
10         2     -3   5.995666  16.456403  40.068459  109.976553
11         2     -2   5.938778  16.309013  40.001627  109.852077
12         2     -1   5.881922  16.161550  39.935050  109.728136
13         2      0   3.660379  10.273038  17.428358   48.913558
14         2      1   3.6

In [54]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

mbdplot = px.bar(combined_validation, x  = "Position", y = "mbd", color = "Offset", barmode = "group", width = 1000, height = 500)
mbdplot.show()

mbdabsplot = px.bar(combined_validation, x  = "Position", y = "mbd_abs", color = "Offset", barmode = "group" , width = 1000, height = 500)
mbdabsplot.show()

rmseplot = px.bar(combined_validation, x  = "Position", y = "rmse", color = "Offset", barmode = "group", width = 1000, height = 500)
rmseplot.show()

rmseabsplot = px.bar(combined_validation, x  = "Position", y = "rmse_abs", color = "Offset", barmode = "group", width = 1000, height = 500)
rmseabsplot.show()

# validplot = px.bar(combined_validation, x  = "Position", y = ["mbd", "mbd_abs", "rmse", "rmse_abs"], color = "Offset", facet_row = y,barmode = "group", width = 1000, height = 500)

fig1 = px.line(combined, x = 'Datetime', y = ["Modeled", "Measured"], facet_row = 'Position', facet_col = "Offset", width = 1000, height = 1000)
fig1.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig1.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig1.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig1.show()


fig2 = px.scatter(combined, x="Modeled", y="Measured", trendline="ols", facet_col = "Offset", facet_row = "Position", hover_data = combined.columns[:], width = 1000, height = 1000)
fig2.show()

fig3 = px.line(combined, x = 'Datetime', y = ['Modeled', 'Measured', 'ModeledMinusMeasured'], facet_row = 'Position', facet_col = "Offset", width = 1000, height = 1000)
fig3.update_traces(opacity = 0.5)
fig3.update_layout(
    plot_bgcolor='white',
    xaxis_title = "",
    yaxis_title = "15-min average irradiance (W/m<sup>2</sup>)",
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5
    ),
    legend_title=dict(
        text = "Data type"
    ),
    legend_orientation="h"
)
fig3.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
fig3.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
fig3.show()

In [55]:
def figures_to_html(figs, filename="dashboard.html"):
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")


figures_to_html([mbdplot, mbdabsplot, rmseplot, rmseabsplot, fig1, fig2, fig3], 'BARN_validation_4inchoffsets.html')

In [14]:
# Filter the DataFrame for specific Position and Offset combinations that yield the lowest error
combined_filter = combined_ref[
    ((combined_ref['Position'] == 1) & (combined_ref['Offset'] == 1)) |
    ((combined_ref['Position'] == 2) & (combined_ref['Offset'] == 4)) |
    ((combined_ref['Position'] == 3) & (combined_ref['Offset'] == -4)) |
    ((combined_ref['Position'] == 4) & (combined_ref['Offset'] == 4)) |
    ((combined_ref['Position'] == 5) & (combined_ref['Offset'] == 4))
].rename(columns = {"GHI":"Background GHI", "Modeled":"Modeled GHI", "Measured":"Measured GHI", "PAR_con":"PAR in control", "PAR_pv":"PAR in array"})

# combined_result = pd.merge(
#     combined_filter,
#     combined_validation,
#     on=["Position", "Offset"],
#     how="outer"  # Use "inner" for matching rows only, or "outer" for including all rows
# )

# result = combined_result[combined_result['Position'] == 3].drop(columns=['Position', 'Offset', 'Center'])
# print(combined_result)
# Print column names and data types
# print(combined_filter)


In [61]:
# # Create the line plot with facets
# fig1 = px.line(
#     combined_filter,
#     x="Datetime",
#     y=["Background GHI", "Modeled ground irradiance", "Measured ground irradiance", "PAR in control"],
#     facet_col="Position",
#     facet_col_wrap=5,
#     labels={"value": "Value", "Datetime": "Datetime"},
#     color_discrete_map={
#         "Background GHI": "rgba(128, 0, 128, 0.4)",  # Purple
#         "Modeled ground irradiance": "rgba(38, 139, 139, 0.7)",  # Teal
#         "Measured ground irradiance": "rgba(255, 127, 0, 0.7)",  # Orange
#         "PAR in control": "rgba(0, 0, 255, 0.7)"  # Blue
#     }
# )

# # Identify the axis IDs for the third facet (Position=3)
# facet_axis_suffix = fig1.data[2].xaxis.replace("x", "")  # Gets the axis suffix for Position=3

# # Add the "PAR_pv" trace specifically to the third facet
# par_pv_data = combined_filter[combined_filter["Position"] == 3]
# fig1.add_trace(
#     go.Scatter(
#         x=par_pv_data["Datetime"],
#         y=par_pv_data["PAR in array"],
#         mode="lines",
#         name="PAR in array",
#         line=dict(color="rgba(255, 0, 255, 0.7)"),  # Magenta
#         # marker=dict(color="rgba(255, 0, 255, 0.7)"),
#         showlegend=True  # Ensure legend entry appears
#     ),
#     row=1,  # Assign explicitly to the correct row for Position=3
#     col=3   # Assign explicitly to the correct column for Position=3
# )

# # # Add a secondary y-axis for Position=5 using annotation
# # fig1.add_annotation(
# #     xref="paper", yref="paper",  # Relative to the entire plot
# #     x=1.02, y=0.5,  # Position the annotation near the right edge, vertically centered for facet 5
# #     text="15-min average PPFD (µmol/m²/s)",  # Annotation text
# #     showarrow=False,
# #     font=dict(size=12),
# #     textangle=90  # Rotate annotation to mimic vertical axis title
# # )


# # Update layout for axis titles and formatting
# fig1.update_layout(
#     xaxis_title=None,  # Remove x-axis title
#     yaxis_title="15-min average irradiance (W/m<sup>2</sup>)<br>15-min average PPFD (µmol/m²/s)",  # Superscript for W/m^2
#     margin={"t": 50, "l": 50, "r": 50, "b": 50},
#     plot_bgcolor="white",
#     height=400,
#     legend=dict(
#         yanchor="top",
#         y=1.3,
#         xanchor="center",
#         x=0.5,
#         orientation="h",
#         title_text="Data type"
#     ),
#     font=dict(size=18),  # Set the default font size for all elements
#     title_font=dict(size=18),  # Title font size
#     legend_font=dict(size=18)  # Legend font size
# )
# fig1.update_xaxes(
#     mirror=True,
#     ticks='outside',
#     showline=True,
#     linecolor='black',
#     gridcolor='lightgrey',
#     title = " "
# )
# fig1.update_yaxes(
#     mirror=True,
#     ticks='outside',
#     showline=True,
#     linecolor='black',
#     gridcolor='lightgrey'
# )


# # Ensure SRRL is a dotted line
# fig1.for_each_trace(lambda trace: trace.update(line_dash="dash") if trace.name == "Background GHI" else None)
# fig1.for_each_trace(lambda trace: trace.update(line_dash="dot") if trace.name == "PAR in array" else None)
# fig1.for_each_trace(lambda trace: trace.update(line_dash="dot") if trace.name == "PAR in control" else None)

# # Remove x-axis title and set tick format for all facets
# fig1.for_each_xaxis(lambda axis: axis.update(title=None, tickformat="%H:%M"))

# # Show the plot
# fig1.show()

# linreg = px.scatter(combined_filter2.reset_index(), x = 'measured', y = 'modeled', facet_col = 'position', facet_col_wrap=5, trendline="ols", trendline_color_override="black", width = figwidth, height = figheight)

parfig = px.scatter(combined_filter2, x="Modeled ground irradiance", y="Measured ground irradiance", trendline="ols", facet_col = "Position", hover_data = combined_filter.columns[:])
parfig.update_layout(
    plot_bgcolor="white",
    height=400,
    margin={"t": 50, "l": 50, "r": 50, "b": 50},
    legend=dict(
        yanchor="top",
        y=1.3,
        xanchor="center",
        x=0.5,
        orientation="h",
        title_text="Data type"
    )
)
parfig.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
parfig.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)

parfig.show()

combined_validation = combined_validation[
    ((combined_validation['Position'] == 1) & (combined_validation['Offset'] == 1)) |
    ((combined_validation['Position'] == 2) & (combined_validation['Offset'] == 4)) |
    ((combined_validation['Position'] == 3) & (combined_validation['Offset'] == -4)) |
    ((combined_validation['Position'] == 4) & (combined_validation['Offset'] == 4)) |
    ((combined_validation['Position'] == 5) & (combined_validation['Offset'] == 4))
]
# .rename(columns = {"GHI":"Background GHI", "Modeled":"Modeled GHI", "Measured":"Measured GHI", "PAR_con":"PAR in control", "PAR_pv":"PAR in array"})

mbdplot = px.bar(combined_validation, x  = "Position", y = "mbd", barmode = "group", width = 1000, height = 500)
mbdplot.update_layout(
    plot_bgcolor="white",
    height=400,
    margin={"t": 50, "l": 50, "r": 50, "b": 50},
    legend=dict(
        yanchor="top",
        y=1.3,
        xanchor="center",
        x=0.5,
        orientation="h",
        title_text="Data type"
    )
)
mbdplot.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
mbdplot.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
mbdplot.show()

mbdabsplot = px.bar(combined_validation, x  = "Position", y = "mbd_abs", barmode = "group" , width = 1000, height = 500)
mbdabsplot.update_layout(
    plot_bgcolor="white",
    height=400,
    margin={"t": 50, "l": 50, "r": 50, "b": 50},
    legend=dict(
        yanchor="top",
        y=1.3,
        xanchor="center",
        x=0.5,
        orientation="h",
        title_text="Data type"
    )
)
mbdabsplot.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
mbdabsplot.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
mbdabsplot.show()

rmseplot = px.bar(combined_validation, x  = "Position", y = "rmse", barmode = "group", width = 1000, height = 500)
rmseplot.update_layout(
    plot_bgcolor="white",
    height=400,
    margin={"t": 50, "l": 50, "r": 50, "b": 50},
    legend=dict(
        yanchor="top",
        y=1.3,
        xanchor="center",
        x=0.5,
        orientation="h",
        title_text="Data type"
    )
)
rmseplot.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
rmseplot.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
rmseplot.show()

rmseabsplot = px.bar(combined_validation, x  = "Position", y = "rmse_abs", barmode = "group", width = 1000, height = 500)
rmseabsplot.update_layout(
    plot_bgcolor="white",
    height=400,
    margin={"t": 50, "l": 50, "r": 50, "b": 50},
    legend=dict(
        yanchor="top",
        y=1.3,
        xanchor="center",
        x=0.5,
        orientation="h",
        title_text="Data type"
    )
)
rmseabsplot.update_xaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey',
    title = " "
)
rmseabsplot.update_yaxes(
    mirror=True,
    ticks='outside',
    showline=True,
    linecolor='black',
    gridcolor='lightgrey'
)
rmseabsplot.show()




ValueError: Value of 'x' is not the name of a column in 'data_frame'. Expected one of ['Datetime', 'Position', 'x', 'Modeled', 'Center', 'Offset', 'Measured', 'PAR_con', 'PAR_pv'] but received: Modeled ground irradiance

In [47]:
import plotly.io as pio
figwidth = 11.8*96
figheight = 4.15*96

fig1.write_image("GHIPAR2.png", format="png", width= figwidth, height = figheight, scale = 1)


In [ ]:
parfig = px.scatter(combined, x="Modeled", y="Measured", trendline="ols", facet_col = "Offset", facet_row = "Position", hover_data = combined.columns[:], width = 1000, height = 1000)

## The following figures are best displayed in wide-screen.

## Stitch all the figures into a single HTML
Function below taken from https://stackoverflow.com/questions/45577255/plot-multiple-figures-as-subplots

In [ ]:
def figures_to_html(figs, filename="dashboard.html"):
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")


figures_to_html([fig1, fig2, fig3, fig4], 'BARN_validation.html')

In [ ]:
print(modeledandmeasured_melt2)

In [ ]:
# Create the line plot with facets
fig1 = px.line(
    combined_filter,
    x="Datetime",
    y=["SRRL", "Modeled", "Measured", "PAR_con"],
    facet_col="Position",
    facet_col_wrap=5,
    labels={"value": "Value", "Datetime": "Datetime"},
    color_discrete_map={
        "SRRL": "rgba(128, 0, 128, 0.4)",  # Purple
        "Modeled": "rgba(38, 139, 139, 0.7)",  # Teal
        "Measured": "rgba(255, 127, 0, 0.7)",  # Orange
        "PAR_con": "rgba(0, 0, 255, 0.7)"  # Blue
    }
)

# Add the "PAR_pv" trace specifically to the third facet (Position=3)
par_pv_data = combined_result[combined_result["Position"] == 3]
fig1.add_trace(
    go.Scatter(
        x=par_pv_data["Datetime"],
        y=par_pv_data["PAR_pv"],
        mode="lines",
        name="PAR_pv",
        line=dict(color="rgba(255, 0, 255, 0.7)"),  # Magenta
        showlegend=True  # Ensure legend entry appears
    )
)

# Add a secondary y-axis for Position=5 using annotation
fig1.add_annotation(
    xref="paper", yref="paper",  # Relative to the entire plot
    x=1.02, y=0.5,  # Position the annotation near the right edge, vertically centered for facet 5
    text="15-min average PPFD (µmol/m²/s)",  # Annotation text
    showarrow=False,
    font=dict(size=12),
    textangle=90  # Rotate annotation to mimic vertical axis title
)

# General layout updates
fig1.update_layout(
    xaxis_title=None,  # Remove x-axis title
    yaxis_title="15-min average irradiance (W/m<sup>2</sup>)",  # Superscript for W/m²
    margin={"t": 50, "l": 50, "r": 100, "b": 50},  # Extra margin for annotation space
    plot_bgcolor="white",
    height=400,
    legend=dict(
        yanchor="top",
        y=1.2,
        xanchor="center",
        x=0.5,
        orientation="h",
        title_text="Data type"
    )
)

# Update x- and y-axes for all facets
fig1.update_xaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
    title=None,
    tickformat="%H:%M"
)
fig1.update_yaxes(
    mirror=True,
    ticks="outside",
    showline=True,
    linecolor="black",
    gridcolor="lightgrey",
    range=[0, 2200]
)

# Ensure SRRL is a dotted line
fig1.for_each_trace(lambda trace: trace.update(line_dash="dash") if trace.name == "SRRL" else None)

# Show the plot
fig1.show()


In [ ]:
type(fig1.layout)

Next steps:<br>
Divide the data by AM and PM and <br>
Implement 15-minute averaging to the data: dataframe.resample(freq='15T', label='right', closed='right').mean() <br>
Keep the data in wide form to avoid repeating indices <br>
Look for the script MBD_RMSE_functions.py and perform error analysis to annotatet the scatterplot (also ) <br>
Plot Irradiance sensor 3 vs PAR sensor (for within row and in control (GHI from SRRL)) and use the time of the day as a gradient and point shape as the data type <br>
Superimpose some of the figures with a side-view of an agrivoltaics row